In [2]:
#dataset creator
def data_set_creator(id):
    import cv2
    import numpy as np
    faccas=cv2.CascadeClassifier("D:\\face_attendence_model\\face_xml_file\\cas.xml")
    cam=cv2.VideoCapture(0)
    
    name=input("Enter your Name: ")
    sample=0
    while True:
        c,f=cam.read()
        g=cv2.cvtColor(f,cv2.COLOR_BGR2GRAY)
        cor=faccas.detectMultiScale(g,scaleFactor=1.05,minNeighbors=5)
        for x,y,w,h in cor:
            sample+=1
            cv2.imwrite("D:\\face_attendence_model\\dataSet\\"+str(id)+"_"+"{}".format(sample)+".jpg",g[y:y+h,x:x+w])
            f=cv2.rectangle(f,(x,y),(x+w,y+h),(255,0,0),3)
            cv2.waitKey(5)
        cv2.imshow("Image recognizer",f)
        if sample>90:
            break
    cam.release()
    cv2.destroyAllWindows()
    d[id]=name
    add_details(name)


In [3]:
def otp_verify(number):
    import random
    from twilio.rest import Client
    otp=random.randint(10000,99999)
    #print(otp)
    acc_sid='ACa1a1b602377d7b750fa8dd86a80c2567'
    auth_token='72a2ab4c2351db50349c79bd1613fe6b'
    client=Client(acc_sid,auth_token)
    msg=client.messages.create(
        body="YOUR OTP IS "+str(otp),
        from_='+12153984633',
        to='+91'+str(number)
    )
    
    check=int(input("Enter your OTP number:"))
    if check==otp:
        print("You are registered sucessfully.")
        return number
    else:
        num=int(input("RE-Enter your number and verify:"))
        otp_verify(num)
        

def create_table():
    import sqlite3
    con=sqlite3.connect('employee.db')
    c=con.cursor()
    c=con.cursor()
    #create table
    c.execute("""CREATE TABLE employe (
    Name text,
    Phone_Number integer,
    Email text
    )""")
    con.commit()
    con.close()
def add_details(name):
    a=input("Enter your mail ID:")
    aa=int(input("Enter your phone number:"))
    numberr=aa
    #numberr=otp_verify(aa)
    
    import sqlite3
    con=sqlite3.connect('employee.db')
    c=con.cursor()
    c.execute("INSERT INTO employe VALUES (?,?,?)",(name,a,numberr))
    con.commit()
    con.close()
def delete():
    import sqlite3
    con=sqlite3.connect('employee.db')
    c=con.cursor()
    c.execute("DROP TABLE employe")
    con.commit()
    con.close()
def fetch():
    import sqlite3
    con=sqlite3.connect('employee.db')
    c=con.cursor()
    c.execute("SELECT rowid, * FROM employe")
    items=c.fetchall()
    for item in items:
        print(item)
    con.commit()
    con.close()
    
#create_table()
#add_details('stonsonn')
#fetch()
#delete()
    

In [4]:
#training the model
def model_train():
    import os
    import cv2
    import numpy as np
    from PIL import Image
    rec = cv2.face.LBPHFaceRecognizer_create()
    path="D:\\face_attendence_model\\dataSet"
    def get(path):
        im=[os.path.join(path,f) for f in os.listdir(path)]
        fac=[]
        IDs=[]
        for i in im:
            facimg=Image.open(i).convert('L')
            facenp=np.array(facimg,'uint8')
            ID=int(os.path.split(i)[-1].split('_')[0])
            fac.append(facenp)
            IDs.append(ID)
            cv2.imshow('Training',facenp)
            cv2.waitKey(10)
        return IDs,fac    

    ids,faci=get(path)
    rec.train(faci,np.array(ids))
    rec.save("D:\\face_attendence_model\\train.yml")
    cv2.destroyAllWindows()

In [5]:
#Recognizer
def recognize():
    import cv2
    l=[0 for i in range(101)]
    import numpy as np
    faccas=cv2.CascadeClassifier("D:\\face_attendence_model\\face_xml_file\\cas.xml")
    cam=cv2.VideoCapture(0)
    rec=cv2.face.LBPHFaceRecognizer_create()
    rec.read("D:\\face_attendence_model\\train.yml")
    ID=0
    s=set()
    takes=0
    font=cv2.FONT_HERSHEY_SIMPLEX
    while True:
        c,f=cam.read()
        g=cv2.cvtColor(f,cv2.COLOR_BGR2GRAY)
        cor=faccas.detectMultiScale(g,scaleFactor=1.05,minNeighbors=5)
        for x,y,w,h in cor:
            f=cv2.rectangle(f,(x,y),(x+w,y+h),(0,255,0),3)
            ID,con=rec.predict(g[y:y+h,x:x+w])
            l[ID]+=1
            s.add(ID)
            cv2.putText(f,str(ID),(x,y+h),font,2,(255,0,0),3)
        cv2.imshow("Image recognizer",f)
        takes+=1
        if takes>40:
            break
        if cv2.waitKey(1)==ord('q'):
            break
    cam.release()
    cv2.destroyAllWindows()
    return (l,s)

def get_max(l,s):
    maxi=0
    for i in s:
        maxi=max(maxi,l[i])
        if maxi==l[i]:
            person=i

    return d[person]
#print(get_max(l,s))

In [7]:
#run this only for resetting the complete list(so fresh new dataset can be created)*******************
#******************************************************************************************************************************
#d={}
#id=1

In [8]:
#feeding new data and training
def add_employee():
    global id
    data_set_creator(id)
    model_train()
    id+=1

In [9]:
#1.create entry or load the entry(create--update the dictionary,)
#mark attendance 
#check for mask
def find_employee():
    tuplee=recognize()
    return get_max(tuplee[0],tuplee[1])
    

In [10]:
#add_employee()#this func to add an employee

In [11]:
#find_employee()#this func to identify an employee

In [12]:
#if new:
#add_employee()
#add data to database
#*****************************************************
#if existing
#attendance=[]
#attendance.append(find_employee())
#mark attendance checking database


In [14]:
#resetting the dictionary***********************************************************
#from collections import defaultdict
#sheet=defaultdict(list)
#*********************************************************

In [15]:
def add_to_dict(x):
    from datetime import datetime
    now=datetime.now()
    
    #x=find_employee()
    if check_mask()==True:
        s='Yes'
    else:
        s='No'
    if x not in sheet['Name']:
        sheet['Name'].append(x)
        dtString = now.strftime('%H:%M:%S')
        sheet['Time'].append(dtString)
        sheet['Protocol'].append(s)
    return sheet

In [16]:
print(sheet)

defaultdict(<class 'list'>, {})


In [17]:
#Create a new excel only at the end . Do only at the end,....
def make_excel(sheet):
    from datetime import date
    today = date.today()
    # dd/mm/YY
    d1 = today.strftime("%d%m%Y")
    import pandas as pd
    df = pd.DataFrame(sheet)
    writer = pd.ExcelWriter('D:\\face_attendence_model\\attendance_on_'+d1+'.xlsx', engine='xlsxwriter')
    df.to_excel(writer,index=False)
    writer.save()

In [18]:
'''1.Clear the dataset folder
2.clear sheet dictionary
3.Make ID count=1
4.clear D dictionary
Available functions:
    
    *add_employee()
    
    *find_employee()
    
    *make_excel(sheet)#makes excel out of the created dictionary
    
    *add_to_dict() #runs to find the person and add details to dictionary'''

'1.Clear the dataset folder\n2.clear sheet dictionary\n3.Make ID count=1\n4.clear D dictionary\nAvailable functions:\n    \n    *add_employee()\n    \n    *find_employee()\n    \n    *make_excel(sheet)#makes excel out of the created dictionary\n    \n    *add_to_dict() #runs to find the person and add details to dictionary'

In [19]:
def check_mask():
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
    from tensorflow.keras.preprocessing.image import img_to_array
    from tensorflow.keras.models import load_model
    from imutils.video import VideoStream
    import numpy as np
    import imutils
    import time
    import cv2
    import os
    validate=0
    def detect_and_predict_mask(frame, faceNet, maskNet):
        # grab the dimensions of the frame and then construct a blob
        # from it
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
            (104.0, 177.0, 123.0))

        # pass the blob through the network and obtain the face detections
        faceNet.setInput(blob)
        detections = faceNet.forward()
        print(detections.shape)

        # initialize our list of faces, their corresponding locations,
        # and the list of predictions from our face mask network
        faces = []
        locs = []
        preds = []

        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the detection
            confidence = detections[0, 0, i, 2]

            # filter out weak detections by ensuring the confidence is
            # greater than the minimum confidence
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # ensure the bounding boxes fall within the dimensions of
                # the frame
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

                # extract the face ROI, convert it from BGR to RGB channel
                # ordering, resize it to 224x224, and preprocess it
                face = frame[startY:endY, startX:endX]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = img_to_array(face)
                face = preprocess_input(face)

                # add the face and bounding boxes to their respective
                # lists
                faces.append(face)
                locs.append((startX, startY, endX, endY))

        # only make a predictions if at least one face was detected
        if len(faces) > 0:
            # for faster inference we'll make batch predictions on *all*
            # faces at the same time rather than one-by-one predictions
            # in the above `for` loop
            faces = np.array(faces, dtype="float32")
            preds = maskNet.predict(faces, batch_size=32)

        # return a 2-tuple of the face locations and their corresponding
        # locations
        return (locs, preds)

    # load our serialized face detector model from disk
    prototxtPath = r"D:\Face-Mask-Detection-master\face_detector\deploy.prototxt"
    weightsPath = r"D:\Face-Mask-Detection-master\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

    # load the face mask detector model from disk
    maskNet = load_model("mask_detector.model")

    # initialize the video stream
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    takes=0
    # loop over the frames from the video stream
    while True:
        # grab the frame from the threaded video stream and resize it
        # to have a maximum width of 400 pixels
        takes+=1
        frame = vs.read()
        frame = imutils.resize(frame, width=400)

        # detect faces in the frame and determine if they are wearing a
        # face mask or not
        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

        # loop over the detected face locations and their corresponding
        # locations
        for (box, pred) in zip(locs, preds):
            # unpack the bounding box and predictions
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred
            
            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"
            if label=="Mask":
                validate+=mask
            elif label=="No Mask":
                validate-=withoutMask
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            # display the label and bounding box rectangle on the output
            # frame
            cv2.putText(frame, label, (startX, startY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if takes>50:
            break
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.stop()
    if validate>0:
        return True
    else:
        return False
#check_mask()

In [30]:
#resetting***********************************************************
#delete()
from collections import defaultdict
sheet=defaultdict(list)
d={}
id=1
#create_table()
#delete the dataset images
#*********************************************************

In [29]:
#new or existing
#if new: add to dataset then find
#else: find and mark attendance, check mask too.
import speech_recognition as sr 
r = sr.Recognizer() 
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
newVoiceRate = 145
engine.setProperty('rate',newVoiceRate)

def talk(t):
    engine.say(t)
    engine.runAndWait()
def listen():
    with sr.Microphone() as source: 
        print(".......")
        audio = r.listen(source)           
        try: 
            text = r.recognize_google(audio) 
            print("you said: " + text) 
        except: 
            text="Google Speech Recognition could not understand audio"
        return text  

talk("Are you a new or an existing employee?")
lis=listen()
if 'new' in lis.split():
    add_employee()
else:
    x=find_employee()
    if x in sheet['Name']:
        talk("Have a cup of coffee. Its free over there.")
    else:
        y=add_to_dict(x)
        talk("Hello "+x+"Good Morning , Nice to meet You.Wish you a great day ahead.")

    

.......
you said: new employee
Enter your Name: rahul
Enter your mail ID:rahulgrd09@
Enter your phone number:9176634913


TwilioRestException: HTTP 400 error: Unable to create record: The number  is unverified. Trial accounts cannot send messages to unverified numbers; verify  at twilio.com/user/account/phone-numbers/verified, or purchase a Twilio number to send messages to unverified numbers.

In [50]:
#print((lis.split()))
#to generate excel(generating attendance)
make_excel(y)

['I', 'am']


In [28]:
fetch()

(1, 'AthulKrishna', 'athulrd8@', '7994335370')
